In [1]:
#r "nuget:Serilog.Formatting.Compact.Reader,1.0.3"
#r "nuget:System.Linq.Async,4.0.0"

Installed package System.Linq.Async version 4.0.0

Installed package Serilog.Formatting.Compact.Reader version 1.0.3

In [1]:
using System.IO;
using Serilog.Events;
using Serilog.Formatting.Compact.Reader;
using System.Collections.Generic;
using System.Linq;

In [1]:
public async IAsyncEnumerable<(DateTimeOffset x, float y)> GetBestFrameCount()
{
    var compactFile = new FileInfo(@"C:\Users\vaido\SourceCode\MrBabyData\FrameRateAnalysis\RogueTD_PerformanceMetricsFrameRates.clef");
    using(var stream = compactFile.OpenText())
    {
        while(stream.Peek() > 0){
            var logevent = Serilog.Formatting.Compact.Reader.LogEventReader.ReadFromString(await stream.ReadLineAsync());
            yield return (
                logevent.Timestamp,
                float.Parse(logevent.Properties["bestDuration"].ToString())
            );
        }
    }
}

public struct PlotDatum{
    public double x {get; set;}
    public double y0 {get; set;}
    public double y1 {get; set;}
}

var readData = await GetBestFrameCount().ToListAsync();
var firstTime = readData.First().x;
var data = readData.Select(i => new PlotDatum{x = (i.x - firstTime).TotalSeconds, y0 = 60/i.y, y1 = 60/i.y + 500}).ToList();
data

index,x,y0,y1
0,0,12929.083984375,13429.083984375
1,1.0063196,10181.3984375,10681.3984375
2,2.009968,10153.3154296875,10653.3154296875
3,3.0142602,10096.5908203125,10596.5908203125
4,4.0176302,10176.0458984375,10676.0458984375
5,5.0228337,10146.447265625,10646.447265625
6,6.02705,9876.8681640625,10376.8681640625
7,7.0322281,8921.26953125,9421.26953125
8,8.033096,8694.8955078125,9194.8955078125
9,9.0381437,9574.111328125,10074.111328125


In [1]:
#!javascript
notebookScope.plot = (sgvSelector, dataVariable) => {
    let dtreeLoader = interactive.configureRequire({
        paths: {
            d3: "https://d3js.org/d3.v6.min"
        }
    });

    dtreeLoader(["d3"], function (d3) {


      var buildFrameRatePlot = (svg, width, height, data) => {
        var layoutMeasures = {
          width : width,
          height : height,
          plotPadding: [48,48,48,48], // top, right, bottom, left
          get plotWidth()
          {
            return layoutMeasures.width - layoutMeasures.plotPadding[1] - layoutMeasures.plotPadding[3]
          },
          get plotHeight()
          {
            return layoutMeasures.height - layoutMeasures.plotPadding[0] - layoutMeasures.plotPadding[2]
          }
        }
    
        // clean clear our svg
        svg.selectAll("g").remove();
      
        // plot container
        let container = svg
        .append("g")
        .attr("height", layoutMeasures.plotHeight)
        .attr("width", layoutMeasures.plotWidth)
        .attr("transform", `translate(${layoutMeasures.plotPadding[0]}, ${layoutMeasures.plotPadding[3]})`);
    
        container.append("rect")
          .attr("height", layoutMeasures.plotHeight)
          .attr("width", layoutMeasures.plotWidth)
          .style("fill", "#F7F7F7")
      
        var y = d3.scaleLinear()
          .domain([0, d3.max(data.flatMap(i => [i.y0, i.y1]))])
          .nice()
          .range([layoutMeasures.plotHeight, 0]);
        var x = d3.scaleLinear()
          .domain([0, d3.max(data, i => i.x)])
          .range([0, layoutMeasures.plotWidth])
        var yAxis = g => g
            .attr("transform",`translate(0,0)`)
            .call(d3.axisLeft(y))
        var xAxis = g => g
            .attr("transform",`translate(0,${layoutMeasures.plotHeight})`)
            .call(d3.axisBottom(x))
        container.append("g")
            .call(yAxis);
        container.append("g")
            .call(xAxis);
      
        // Line - best framerate in sample
        var linePoints_BestFramerate = d3.line()
            .x(d => x(d.x))
            .y(d => y(d.y0))
            (data);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'green')
          .attr('stroke-width', 3)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_BestFramerate);
      
        // Line - worst Framerate in sample
          var linePoints_WorstFramerate = d3.line()
            .x(d => x(d.x))
            .y(d => y(d.y1))
            (data);
        container.append('path')
          .attr('fill', 'none')
          .attr('stroke', 'Red')
          .attr('stroke-width', 3)
          .attr("stroke-linejoin", "round")
          .attr('d', linePoints_WorstFramerate);
      
        // Area - difference between best and worst framerate
        var areaDifference = d3.area()
          .curve(d3.curveLinear)
          .defined(d => !isNaN(d.x))
          .defined(d => !isNaN(d.y0))
          .x(d => x(d.x))
          .y0(d => y(d.y0))
          .y1(d => y(d.y1));
        container.append("path")
          .datum(data)
          .attr("fill", "steelblue")
          .attr("d", areaDifference);
      
      // Plot title
      svg.append('text')
        .text("Frame Rate Over Time")
        .attr('x', layoutMeasures.width / 2)
        .attr('y', 24)
        .attr('dominant-baseline', 'middle')
        .attr("text-anchor", "middle")
        .style('font-size', 24)
      
      // X-axis title
      container.append('text')
        .text('Time since scene start (seconds)')
        .attr('dominant-baseline', 'middle')
        .attr('text-anchor', 'middle')
        .attr('x', layoutMeasures.plotWidth / 2)
        .attr('y', layoutMeasures.plotHeight + 36)
      
      // Y-axis title
      container
        .append('g')
        .attr('transform', `translate(-36, ${layoutMeasures.plotHeight / 2})`)
        .append('text')
          .text('Frames per second')
          .attr('dominant-baseline', 'middle')
          .attr('text-anchor', 'middle')
          .attr('transform', 'rotate(-90)')
    }

    var svg = d3.select(sgvSelector);
    console.log(svg);
    interactive.csharp.getVariable(dataVariable)
        .then( data => buildFrameRatePlot(svg, 1200, 700, data));
    });
}

In [1]:
#!html
<svg id="dataPlot1" width=1200 height=700></svg>

#!js
notebookScope.plot("svg#dataPlot1", "data" );

In [1]:
#!html


In [1]:
#!html
